# This notebook describes the process of cleaning the original PISA 2012 dataset in order to use it in my analysis.



## Preliminary Wrangling

In [1]:
#import needed package
import pandas as pd

- By opening the "pisadict_2012.csv" file in Excel I found that there are 635 columns in the original dataset. The file contains a list of those columns along with their descriptions. I used this information in order to load only the columns that I needed for my analysis (which I'll describe in the exploration notebook ("Data_Exploration.ipynb").

In [2]:
#reading the dataset into a dataframe using only the needed columns
pisa_2012 = pd.read_csv("data/pisa_2012.csv", encoding='latin-1', 
                        usecols= ['CNT','STIDSTD','ST04Q01','PV1MATH', 'PV1SCIE','ST11Q01','ST11Q02',
                                  'ST11Q03','ST11Q04','ST57Q05','HISCED'],
                        dtype={21:str, 22:str, 23:str, 24:str}
                       )

In [3]:
#checking if the columns loaded correctly
pisa_2012.columns

Index(['CNT', 'STIDSTD', 'ST04Q01', 'ST11Q01', 'ST11Q02', 'ST11Q03', 'ST11Q04',
       'ST57Q05', 'HISCED', 'PV1MATH', 'PV1SCIE'],
      dtype='object')

In [4]:
#rename the columns using meaningfull names
pisa_2012.rename({'STIDSTD':'Student_Id', 'ST04Q01':'Gender','CNT':'Country',
                  'PV1MATH':'Math_Score','PV1SCIE':'Science_Score','ST11Q01':'Mother_Present', 'ST11Q02':'Father_Present',
                  'ST11Q03':'Brother(s)_Present','ST11Q04':'Sister(s)_Present','ST57Q05':'Study_Time_With_Parent',
                  'HISCED':'Parent_Highest_Education'
                 }, axis='columns', inplace=True)

##### Checking various aspects of the dataset

In [5]:
pisa_2012.columns

Index(['Country', 'Student_Id', 'Gender', 'Mother_Present', 'Father_Present',
       'Brother(s)_Present', 'Sister(s)_Present', 'Study_Time_With_Parent',
       'Parent_Highest_Education', 'Math_Score', 'Science_Score'],
      dtype='object')

In [6]:
pisa_2012.shape

(485490, 11)

In [7]:
pisa_2012.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485490 entries, 0 to 485489
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Country                   485490 non-null  object 
 1   Student_Id                485490 non-null  int64  
 2   Gender                    485490 non-null  object 
 3   Mother_Present            460559 non-null  object 
 4   Father_Present            441036 non-null  object 
 5   Brother(s)_Present        400076 non-null  object 
 6   Sister(s)_Present         390768 non-null  object 
 7   Study_Time_With_Parent    289502 non-null  float64
 8   Parent_Highest_Education  473091 non-null  object 
 9   Math_Score                485490 non-null  float64
 10  Science_Score             485490 non-null  float64
dtypes: float64(3), int64(1), object(7)
memory usage: 40.7+ MB


In [8]:
pisa_2012.head()

,Country,Student_Id,Gender,Mother_Present,Father_Present,Brother(s)_Present,Sister(s)_Present,Study_Time_With_Parent,Parent_Highest_Education,Math_Score,Science_Score
0,Albania,1,Female,Yes,Yes,Yes,Yes,NaN,"ISCED 3A, ISCED 4",406.8469,341.7009
1,Albania,2,Female,Yes,Yes,NaN,Yes,0.0,"ISCED 5A, 6",486.1427,548.9929
2,Albania,3,Female,Yes,Yes,No,Yes,2.0,"ISCED 5A, 6",533.2684,499.6643
3,Albania,4,Female,Yes,Yes,No,Yes,0.0,"ISCED 5A, 6",412.2215,438.6796
4,Albania,5,Female,Yes,Yes,Yes,NaN,NaN,"ISCED 3A, ISCED 4",381.9209,361.5628


In [9]:
pisa_2012.describe()

,Student_Id,Study_Time_With_Parent,Math_Score,Science_Score
count,485490.000000,289502.000000,485490.000000,485490.000000
mean,6134.066201,1.213363,469.621653,475.769824
std,6733.144944,2.353292,103.265391,101.464426
min,1.000000,0.000000,19.792800,2.648300
25%,1811.000000,0.000000,395.318600,404.457300
50%,3740.000000,0.000000,466.201900,475.699400
75%,7456.000000,2.000000,541.057800,547.780700
max,33806.000000,30.000000,962.229300,903.338300


#### From the above results I can identify the follwing problems that need to be addressed before I start my analysis.
>- The 'Study_Time_With_Parent' column contains 289502 rows with NULL values (which is the column with almost 50% of missing values)
>- The 'Study_Time_With_Parent' is of type float instead of int
>- The columns 'Mother_Present' and 'Father_Present' could be in one column (Parent_Present)
>- The columns 'Brother(s)_Present' and 'Sister(s)_Present' could be in one column (Sibling_Present)
>- Calculate a mean score for the 'Math_Score' and 'Science_Score' columns
>- The columns Gender, Parent_Present, Sibling_Present and Parent_Highest_Education should be categories

#### Start cleaning the dataset

In [10]:
# making a copy of the original dataset
pisa_final = pisa_2012.copy()

In [11]:
# I decided to replace the missing values of  'Study_Time_With_Parent' with the mean value
# of that column in order not to lose significant amount of data
pisa_final.Study_Time_With_Parent.fillna(pisa_final.Study_Time_With_Parent.mean(), inplace=True)

In [12]:
pisa_final.loc[:,'Study_Time_With_Parent'] = pisa_final['Study_Time_With_Parent'].astype('int64')

In [13]:
# As for the other missing values I'll drop the coressponding rows
pisa_final = pisa_final.dropna(subset=['Mother_Present', 'Father_Present','Brother(s)_Present', 'Sister(s)_Present',
                                     'Parent_Highest_Education'])

In [14]:
pisa_final.shape

(347783, 11)

In [15]:
# Merging the 'Mother_Present' and 'Father_Present' into a single column 'Parents_Present'
pisa_final['Parents_Present'] = ((pisa_final['Mother_Present'] == 'No') | (pisa_final['Father_Present'] == 'No')).replace(False, 'Both_Parents').replace(True, 'One_Parent')
pisa_final = pisa_final.drop(['Mother_Present','Father_Present'], axis=1)

In [16]:
# Merging the 'Brother(s)_Present' and 'Sister(s)_Present' into a single column 'Siblings_Present'
pisa_final['Siblings_Present'] = ((pisa_final['Brother(s)_Present'] == 'No') | (pisa_final['Sister(s)_Present'] == 'No')).replace(False, 'Yes').replace(True, 'No')
pisa_final = pisa_final.drop(['Brother(s)_Present','Sister(s)_Present'], axis=1)

In [17]:
pisa_final.columns

Index(['Country', 'Student_Id', 'Gender', 'Study_Time_With_Parent',
       'Parent_Highest_Education', 'Math_Score', 'Science_Score',
       'Parents_Present', 'Siblings_Present'],
      dtype='object')

In [18]:
# Calculate the mean score of Math and Science
pisa_final['Mean_Score'] = (pisa_final['Math_Score'] + pisa_final['Science_Score']) / 2
pisa_final.head()

,Country,Student_Id,Gender,Study_Time_With_Parent,Parent_Highest_Education,Math_Score,Science_Score,Parents_Present,Siblings_Present,Mean_Score
0,Albania,1,Female,1,"ISCED 3A, ISCED 4",406.8469,341.7009,Both_Parents,Yes,374.27390
2,Albania,3,Female,2,"ISCED 5A, 6",533.2684,499.6643,Both_Parents,No,516.46635
3,Albania,4,Female,0,"ISCED 5A, 6",412.2215,438.6796,Both_Parents,No,425.45055
5,Albania,6,Female,1,"ISCED 3B, C",396.3312,384.3156,Both_Parents,No,390.32340
6,Albania,7,Female,0,"ISCED 5A, 6",438.0823,508.1499,Both_Parents,No,473.11610


In [19]:
# converting the following columns: Gender, Parent_Present, Sibling_Present, Parent_Highest_Education,
# Parent_Highest_Occupation and Immigration_Status into categories categories

gender_dtype = pd.api.types.CategoricalDtype(categories=['Female','Male'])
pisa_final.loc[:,'Gender'] = pisa_final.Gender.astype(gender_dtype)

parents_present_dtype = pd.api.types.CategoricalDtype(categories=['Both_Parents','One_Parent'])
pisa_final.loc[:,'Parents_Present'] = pisa_final['Parents_Present'].astype(parents_present_dtype)


siblings_present_dtype = pd.api.types.CategoricalDtype(categories=['Yes','No'])
pisa_final.loc[:,'Siblings_Present'] = pisa_final['Siblings_Present'].astype(siblings_present_dtype)


parent_education_dtype = pd.api.types.CategoricalDtype(
                    categories=['None','ISCED 1','ISCED 2','ISCED 3B, C','ISCED 3A, ISCED 4','ISCED 5B','ISCED 5A, 6'],
                    ordered=True)
pisa_final.loc[:,'Parent_Highest_Education'] = pisa_final['Parent_Highest_Education'].astype(parent_education_dtype)


pisa_final.dtypes

Country                       object
Student_Id                     int64
Gender                      category
Study_Time_With_Parent         int64
Parent_Highest_Education    category
Math_Score                   float64
Science_Score                float64
Parents_Present             category
Siblings_Present            category
Mean_Score                   float64
dtype: object

In [20]:
# finally I'll save the dataset in an external csv file for future reference
pisa_final.to_csv("data/pisa2012_final.csv", index=False, encoding='latin-1')

## Continue to "Data_Exploration" notebook for the main analysis of the dataset